<a href="https://colab.research.google.com/github/gkteco/feel-the-agi/blob/main/PDF_to_BQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-cloud-storage google-cloud-bigquery google-cloud-vision google-cloud-documentai



In [ ]:
from google.colab import auth
auth.authenticate_user()

# Set your Google Cloud project
project_id = 'ce-demo-space'
!gcloud config set project {project_id}

# Verify the active project
!gcloud config list project


Updated property [core/project].
[core]
project = ce-demo-space

Your active configuration is: [default]


In [ ]:
import os
from google.cloud import documentai_v1 as documentai
from google.cloud import storage
from google.cloud import bigquery
from google.api_core.exceptions import NotFound

# Set the bucket name and dataset/table names
bucket_name = 'georgia-demo-pdfs'
dataset_name = 'collins_ds'
text_table_name = 'pdf_texts'
project_id = 'ce-demo-space'  # project ID
location = 'us'  # Location for the Document AI processor
processor_id = '3bab9833052f79c5'  # Correct processor ID

# Initialize the clients with the project ID
storage_client = storage.Client(project=project_id)
bigquery_client = bigquery.Client(project=project_id)
documentai_client = documentai.DocumentProcessorServiceClient()


dataset_ref = bigquery_client.dataset(dataset_name)
try:
    bigquery_client.get_dataset(dataset_ref)
except NotFound:
    dataset = bigquery.Dataset(dataset_ref)
    dataset.location = "US"
    bigquery_client.create_dataset(dataset)
    print(f"Dataset {dataset_name} created.")


text_table_ref = dataset_ref.table(text_table_name)
try:
    text_table = bigquery_client.get_table(text_table_ref)
except NotFound:
    text_schema = [
        bigquery.SchemaField('filename', 'STRING'),
        bigquery.SchemaField('content', 'STRING')
    ]
    text_table = bigquery.Table(text_table_ref, schema=text_schema)
    text_table = bigquery_client.create_table(text_table)
    print(f"Table {text_table_name} created in dataset {dataset_name}.")

# Get a list of PDF files in the bucket
pdf_files = []
for blob in storage_client.bucket(bucket_name).list_blobs():
    if blob.name.endswith('.pdf'):
        pdf_files.append(blob.name)

# Loop through the PDF files and extract text content
for pdf_file in pdf_files:
    gcs_source_uri = f"gs://{bucket_name}/{pdf_file}"

    gcs_document = documentai.GcsDocument(gcs_uri=gcs_source_uri, mime_type='application/pdf')
    input_config = documentai.BatchDocumentsInputConfig(gcs_documents=documentai.GcsDocuments(documents=[gcs_document]))

    # Specify the location and the processor ID
    name = f"projects/{project_id}/locations/{location}/processors/{processor_id}"

    request = documentai.BatchProcessRequest(
        name=name,
        input_documents=input_config,
        document_output_config=documentai.DocumentOutputConfig(
            gcs_output_config=documentai.DocumentOutputConfig.GcsOutputConfig(
                gcs_uri=f"gs://{bucket_name}/output/"
            )
        )
    )

    # Process the document
    operation = documentai_client.batch_process_documents(request=request)

    # Wait for the operation to complete
    print(f'Waiting for the operation to finish for {pdf_file}...')
    operation.result(timeout=300)

    # Get the result from GCS
    text_content = ""
    output_prefix = f'output/{pdf_file.split(".")[0]}_output/'
    for blob in storage_client.bucket(bucket_name).list_blobs(prefix=output_prefix):
        if '.json' in blob.name:
            json_string = blob.download_as_string()
            document = documentai.Document.from_json(json_string)
            for page in document.pages:
                for paragraph in page.paragraphs:
                    text_content += paragraph.layout.text + '\n'

    # Insert the text content into BigQuery
    text_dict = {
        'filename': pdf_file,
        'content': text_content
    }
    errors = bigquery_client.insert_rows_json(text_table_ref, [text_dict])
    if errors == []:
        print(f"Inserted text content for {pdf_file} into BigQuery.")
    else:
        print(f"Failed to insert text content for {pdf_file}: {errors}")

print('Text content processing complete.')


Waiting for the operation to finish for 2011 - OER.pdf...


PermissionDenied: 403 Cloud Document AI API has not been used in project 522309567947 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/documentai.googleapis.com/overview?project=522309567947 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry. [links {
  description: "Google developers console API activation"
  url: "https://console.developers.google.com/apis/api/documentai.googleapis.com/overview?project=522309567947"
}
, reason: "SERVICE_DISABLED"
domain: "googleapis.com"
metadata {
  key: "consumer"
  value: "projects/522309567947"
}
metadata {
  key: "service"
  value: "documentai.googleapis.com"
}
]

In [ ]:
!gcloud config list project



[core]
project = ce-demo-space

Your active configuration is: [default]
